# HW3 - Q3 [35 pts]

In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1666880362476_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [2]:
#export
from pyspark.sql.functions import col
from pyspark.sql import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'

    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return

    input_bucket = "s3://cse6242-hw3-q3"

    # Load Trip Data
    trip_path = '/' + size + '/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path,
                           header=True,
                           inferSchema=True)
    print("Trip Count: ", trips.count()
         )  # Prints # of trips (# of records, as each record is one trip)

    # Load Lookup Data
    lookup_path = '/' + size + '/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path,
                            header=True,
                            inferSchema=True)

    return trips, lookup


def main(size, bucket):
    # Runs your functions implemented above.

    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp, lookup)

    # Outputs the results for you to visually see
    final.show()

    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [5]:
#export
def long_trips(trips):
    # Returns a Dataframe (trips) with Schema the same as :trips:
    trips = trips.filter(trips["trip_distance"] >= 2)
    return trips

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [39]:
#export
def manhattan_trips(trips, lookup):
    # Returns a Dataframe (mtrips) with Schema: DOLocationID, pcount
    mtrips = trips.join(lookup, trips["DOLocationID"] == lookup["LocationID"])
    mtrips = mtrips.filter(mtrips["Borough"] == "Manhattan")
    mtrips = mtrips.withColumn('passenger_count',
                               mtrips.passenger_count.cast('int'))
    mtrips = mtrips.groupby("DOLocationID").sum("passenger_count").orderBy(
        col("sum(passenger_count)").desc()).withColumnRenamed(
            "sum(passenger_count)", "pcount").limit(20)

    return mtrips

    #pass

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [21]:
#export
def weighted_profit(trips, mtrips):
    # Returns a Dataframe (wp) with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe

    mtrips = mtrips.withColumnRenamed("DOLocationID", "DOLocationID_mtrips")

    wp = trips.join(mtrips, trips["DOLocationID"] == mtrips["DOLocationID_mtrips"], 'left')  #.groupby('PULocationID').agg({'DOLocationID':'count'})

    wp = wp.groupby("PULocationID").agg({
        'total_amount': 'avg',
        'tpep_pickup_datetime': 'count',
        'DOLocationID_mtrips': 'count'
    })

    wp = wp.withColumnRenamed(
        'count(DOLocationID_mtrips)', 'count_popular').withColumnRenamed(
            'count(tpep_pickup_datetime)', 'count_total').withColumnRenamed(
                'count(tpep_pickup_datetime)',
                'count_total').withColumnRenamed('avg(total_amount)',
                                                 'total_average')

    wp = wp.withColumn('count_popular',
                       wp.count_popular.cast('int')).withColumn(
                           'count_total',
                           wp.count_total.cast('int')).withColumn(
                               'total_average',
                               wp.total_average.cast('decimal(38, 10)'))

    wp = wp.withColumn('proportion_of_popular',
                       wp.count_popular / wp.count_total)

    wp = wp.withColumn('weighted_profit',
                       wp.proportion_of_popular * wp.total_average)

    wp = wp.drop('count_popular', 'total_average', 'count_total',
                 'proportion_of_popular')

    return wp
    #pass

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [48]:
#export
def final_output(wp, lookup):
    # Returns a Dataframe (final) with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe

    final = wp.join(lookup, wp["PULocationID"] == lookup["LocationID"]).orderBy(
        col("weighted_profit").desc())

    final = final.select('Zone', 'Borough', 'weighted_profit')

    final = final.limit(20)

    final = final.coalesce(1)

    return final
    #pass

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading.
</div>

In [20]:
#wp = weighted_profit(trips, mtrips)

#wp.show(5)

#final_output(wp, lookup).show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [162]:
#lookup.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Interrupted by user


In [49]:
# trips, lookup = load_data()
# trips = long_trips(trips)
# mtrips = manhattan_trips(trips, lookup)
# wp = weighted_profit(trips, mtrips)
final_output(wp, lookup).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------+------------------+
|                Zone|      Borough|   weighted_profit|
+--------------------+-------------+------------------+
|Arrochar/Fort Wad...|Staten Island|31.784444444444436|
|     Freshkills Park|Staten Island|29.342499999999998|
|       Port Richmond|Staten Island|24.357777777777777|
|Flushing Meadows-...|       Queens|24.330137731666976|
|        Baisley Park|       Queens| 23.62469433349845|
|     Randalls Island|    Manhattan| 23.12457751514923|
|       South Jamaica|       Queens|22.174114939432254|
|     Mariners Harbor|Staten Island|21.786556927297667|
|Saint George/New ...|Staten Island|21.491893491124262|
|       Arden Heights|Staten Island|          20.29625|
|        Astoria Park|       Queens|19.965360000000004|
|   LaGuardia Airport|       Queens|17.923153949607126|
|  Murray Hill-Queens|       Queens|17.585450424382717|
|         JFK Airport|       Queens|16.958978201179246|
|             Jamaica|       Queens|14.879835188

In [40]:
manhattan_trips(trips, lookup).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------+
|DOLocationID|pcount|
+------------+------+
|         236|147086|
|         161|117642|
|         230|112155|
|         239|104164|
|         162|101209|
|         238|100189|
|          48| 98784|
|         170| 96109|
|         142| 95053|
|         231| 94621|
|         263| 94548|
|          79| 94421|
|         237| 90335|
|         141| 90061|
|         140| 84212|
|          68| 79559|
|         234| 75786|
|         186| 73863|
|         163| 73145|
|          13| 70693|
+------------+------+

In [41]:
main('large', 's3://cse6242-mgrillo3/output-large')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

mgrillo3
Trip Count:  187203269
+--------------------+-------------+---------------+
|                Zone|      Borough|weighted_profit|
+--------------------+-------------+---------------+
|        Baisley Park|       Queens|  29.3604557791|
|Flushing Meadows-...|       Queens|  27.3048457336|
|       South Jamaica|       Queens|  26.2949162399|
|     Randalls Island|    Manhattan|  24.1509899402|
|        Astoria Park|       Queens|  21.7064171121|
|Briarwood/Jamaica...|       Queens|  19.9450646318|
|Springfield Garde...|       Queens|  19.4683092888|
|             Jamaica|       Queens|  19.2839430001|
|              Corona|       Queens|  18.2287692482|
|   LaGuardia Airport|       Queens|  18.1813388084|
|         Jamaica Bay|       Queens|  17.1005294468|
|             Maspeth|       Queens|  17.0054506401|
|Eltingville/Annad...|Staten Island|  16.8377647569|
|         JFK Airport|       Queens|  16.7777253482|
|        Battery Park|    Manhattan|  12.8497803111|
| Morningside 

In [12]:
#mtrips = manhattan_trips(trips, lookup)

wp = weighted_profit(trips, mtrips)

#FUNCIONO NO SE PORQUE PERO NO IMPORTA JAAAAAA

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…